In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from NQE_class import NQE
from NQE_train_class import NQE_Train
from RNN_class import RNNModel, RNN_layer

In [2]:
n_qu = 5

## Data uploading

In [3]:
locations = ['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland', 'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond', 'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru', 'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera']

In [4]:
num_data = 3000

In [5]:
train_data_dict = dict()
label_data_dict = dict()
for e in locations:
    # print(e)
    train_df = pd.read_csv("./data/train_data_" + e + ".csv")
    label_df = pd.read_csv("./data/label_data_" + e + ".csv")
    train_data_dict[e] = torch.tensor(train_df[["MinTemp","MaxTemp","Rainfall","Humidity3pm","Pressure3pm"]].to_numpy()[:num_data]).to(torch.float)
    label_data_dict[e] = torch.tensor(label_df['RainTomorrow'].to_numpy()[:num_data]).to(torch.float)


In [6]:
for e in label_data_dict:
    print(e, list(label_data_dict[e]).count(0))


Adelaide 100
Albany 20
Albury 29
AliceSprings 7
BadgerysCreek 81
Ballarat 12
Bendigo 5
Brisbane 27
Cairns 52
Canberra 16
Cobar 21
CoffsHarbour 56
Dartmoor 66
Darwin 0
GoldCoast 60
Hobart 5
Katherine 19
Launceston 8
Melbourne 758
MelbourneAirport 0
Mildura 2
Moree 155
MountGambier 6
MountGinini 133
Newcastle 84
Nhil 9
NorahHead 75
NorfolkIsland 45
Nuriootpa 6
PearceRAAF 246
Penrith 74
Perth 0
PerthAirport 0
Portland 13
Richmond 58
Sale 9
SalmonGums 46
Sydney 7
SydneyAirport 4
Townsville 6
Tuggeranong 41
Uluru 57
WaggaWagga 33
Walpole 187
Watsonia 10
Williamtown 456
Witchcliffe 57
Wollongong 48
Woomera 18


In [7]:
train_data_dict['Darwin'].shape

torch.Size([3000, 5])

In [8]:
x_train = train_data_dict['Darwin']
y_train = label_data_dict['Darwin']

non_zero = (y_train != 0).nonzero()
print(len(non_zero))

x_train = torch.squeeze(train_data_dict['Darwin'][non_zero])
y_train = torch.squeeze(label_data_dict['Darwin'][non_zero])

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x_train , y_train, test_size = 0.2, random_state = 42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

3000


C:\Users\pop75\AppData\Local\Temp\ipykernel_4272\2769686528.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
C:\Users\pop75\AppData\Local\Temp\ipykernel_4272\2769686528.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [9]:
total_feature_data = torch.concat([X_train_tensor, X_test_tensor], dim = 0).reshape((-1, 5, 5))
total_label_data = torch.concat([y_train_tensor, y_test_tensor], dim = 0).reshape((-1, 5))

# total_feature_data.reshape((600, 5, 5))
# total_label_data.reshape((600, 5))


print(total_feature_data.shape)
print(total_label_data.shape)

torch.Size([600, 5, 5])
torch.Size([600, 5])


In [10]:
seq_data = data_seq(total_feature_data, total_label_data)
train_loader, test_loader = seq_data.split_data(test_ratio=0.2, batch_size=32)

In [11]:
for e in train_loader:
    print(len(e))
    print(len(e[0]))
    print(len(e[0][0]))
    print(len(e[0][0][0]))
    # print(len(e[0][0][0][0])) # Err
    break

2
32
5
5


## NQE Train

In [12]:
fc_nqe = NQE(n_qu, 'FC')
fc_nqe.load_state_dict(torch.load("./models/nqe_fc_loss220.pth"))

kan_nqe = NQE(n_qu, 'KAN')
kan_nqe.load_state_dict(torch.load("./models/nqe_kan_loss284.pth"))

<All keys matched successfully>

## Model, NQE model, criterion, optimizer

In [13]:
from RNN_class import RNN_layer

In [14]:
no_nqe_RNN = RNN_layer(5, 1, 5)
fc_nqe_RNN = RNN_layer(5, 1, 5, nQE_model = fc_nqe)
kan_nqe_RNN = RNN_layer(5,1, 5, nQE_model = kan_nqe)

criterion = nn.MSELoss()
optimizer1 = optim.Adam(no_nqe_RNN.parameters(), lr = 0.02)
optimizer2 = optim.Adam(fc_nqe_RNN.parameters(), lr = 0.02)
optimizer3 = optim.Adam(kan_nqe_RNN.parameters(), lr = 0.02)


## Train

In [15]:
train_corpus = train_seq(no_nqe_RNN, train_loader, test_loader)

train_loss_list, test_loss_list = train_corpus.train(500, optimizer1, criterion)

c:\Users\pop75\anaconda3\envs\EMT\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 5])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


 loss : 0.81391



c:\Users\pop75\anaconda3\envs\EMT\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([120, 5])) that is different to the input size (torch.Size([120, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


 train_loss : 0.81391 test_loss : 0.75756

 loss : 0.80597

 train_loss : 0.80597 test_loss : 0.76237

 loss : 0.79552

 train_loss : 0.79552 test_loss : 0.76163

 loss : 0.78625

 train_loss : 0.78625 test_loss : 0.75670

 loss : 0.78406

 train_loss : 0.78406 test_loss : 0.75659

 loss : 0.74727

KeyboardInterrupt: 

## Plotting

In [78]:
import matplotlib.pyplot as plt

plt.plot(train_loss_list, label = 'train loss')
plt.ylim(0,0.04)
plt.plot(test_loss_list, label = 'test loss')
plt.ylabel('loss')
plt.xlabel('iteration')
plt.legend()

NameError: name 'train_loss_list' is not defined

## MSE

In [63]:
def decoding(input,max_value):
  max = 1754.882
  min = 85.192
  return (max-min)/max_value*input+min

for feature,label in test_loader:
    pred = decoding(model(feature),1)
    true = decoding(label,1)
    print(nn.MSELoss()(true,pred))
    break

tensor(22270.1816, grad_fn=<MseLossBackward0>)


예시 : pd.DataFrame{’tain_loss’:train_loss,…..}

저장할때 index = False 해야함

data.to_csv(’경로명’,index=False)

폴더명 :  result_data

파일명 : <model_name>_<method_name>_data.csv

In [64]:
pred_list, train_label_list, test_pred_list, test_label_list = get_data(model=model, train_loader=test_loader, test_loader=test_loader)

data_df = pd.DataFrame({'train_loss': train_loss_list, 'test_loss':test_loss_list,'train_predict':pred_list,'train_label':train_label_list,'validation_pred':test_pred_list,'validation_label':test_label_list})

In [65]:
print(data_df)

                       0                                                  1
0             train_loss  [0.7028571963310242, 0.26157712936401367, 0.05...
1         test_loss_list  [0.4050746560096741, 0.09366366267204285, 0.01...
2          train_predict  [0.5189198, 0.63135177, 0.5798306, 0.6479473, ...
3       train_label_list  [0.6068493, 0.6901615, 0.7628788, 0.8246914, 0...
4   validation_pred_list  [0.5189198, 0.63135177, 0.5798306, 0.6479473, ...
5  validation_label_list  [0.6068493, 0.6901615, 0.7628788, 0.8246914, 0...


In [66]:
data_df.to_csv('./result_data_ysj/qRNN_NQEKAN_mse22270._data.csv',index=False)